Data source: https://www.encodeproject.org/files/ENCFF780PGS/

In [2]:
import pandas as pd
import numpy as np
from utils import *
import numpy as np
import openmm as mm
import openmm.unit as u
from tqdm import tqdm
from sys import stdout
from mdtraj.reporters import HDF5Reporter
from openmm.app import PDBFile, PDBxFile, ForceField, Simulation, PDBReporter, PDBxReporter, DCDReporter, StateDataReporter, CharmmPsfFile,  DCDFile
import random
import pyvista as pv
import mdtraj as md
from initial_structures_defs import *

In [25]:
df = pd.read_csv("data/ENCFF780PGS.bedpe", 
                 sep="\t", 
                 header= None)
df.columns = ["chrom1", "start1", "end1", "chrom2","start2", "end2", "score"]

In [26]:
df.head()

,chrom1,start1,end1,chrom2,start2,end2,score
0,chr10,73579,74148,chr10,205258,205891,4
1,chr10,76245,76897,chr10,454409,455026,3
2,chr10,72811,73533,chr10,122997924,122998466,5
3,chr10,88651,89594,chr10,109368,110589,3
4,chr10,113948,114691,chr10,179975,180632,3


In [27]:
df_chr1 = df[df['chrom1'] == 'chr1']

In [28]:
df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2

/var/folders/gp/n7ttdtg56pg0s6ydv9zj9wth0000gn/T/ipykernel_809/314167720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle1'] = (df_chr1['end1'] + df_chr1['start1'])/2
/var/folders/gp/n7ttdtg56pg0s6ydv9zj9wth0000gn/T/ipykernel_809/314167720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chr1['middle2'] = (df_chr1['end2'] + df_chr1['start2'])/2


Scaling for 10 connections

In [29]:
middle_points = df_chr1[['middle1', 'middle2']].reset_index(drop=True)
middle_points


,middle1,middle2
0,605183.5,942849.5
1,779228.5,825798.0
2,828688.0,869860.0
3,832697.5,858533.5
4,865574.5,905542.5
...,...,...
117707,248899548.0,248924561.5
117708,248909067.5,248922497.5
117709,248915281.5,248923600.5
117710,248915281.5,248924765.0


In [38]:
def scale_value(x, original_min, original_max, new_min, new_max):
    return new_min + ((x - original_min) * (new_max - new_min) / (original_max - original_min))


def scale_bead_chain(og_middle_points, nr_connections, new_min=1, new_max=200):
    trimmed_df = og_middle_points.head(nr_connections)
    original_min = np.min(trimmed_df)
    original_max = np.max(trimmed_df)
    df_scaled = trimmed_df.map(lambda x: scale_value(x, original_min, original_max, new_min, new_max))
    df_scaled = df_scaled.map(lambda x: int(x))
    
    return df_scaled


In [39]:
df_scaled = scale_bead_chain(middle_points, 10)

In [40]:
df_scaled

,middle1,middle2
0,1,178
1,92,117
2,118,140
3,120,134
4,138,159
5,137,177
6,140,146
7,140,152
8,139,186
9,140,200


In [41]:
# 0. Generate some initial structure
N_beads=200
points = helisa(N_beads)
write_mmcif(points,'init_struct.cif')
generate_psf(N_beads,'LE_init_struct.psf')

# 1. Define System
pdb = PDBxFile('init_struct.cif')
forcefield = ForceField('forcefields/classic_sm_ff.xml')
system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

# 2. Define the forcefield
# 2.1. Harmonic bond borce between succesive beads
bond_force = mm.HarmonicBondForce()
system.addForce(bond_force)
for i in range(system.getNumParticles() - 1):
    bond_force.addBond(i, i + 1, 0.1, 300000.0)

# Connecting selected beads
for i in range(len(df_scaled)):
    middle1, middle2 = df_scaled.iloc[i,:]
    bond_force.addBond(middle1-1, middle2-1, 0.001, 0.001)


#2.2. Harmonic angle force between successive beads so as to make chromatin rigid
angle_force = mm.HarmonicAngleForce()
system.addForce(angle_force)
for i in range(system.getNumParticles() - 2):
    angle_force.addAngle(i, i + 1, i + 2, np.pi, 0.001)
    
# 3. Minimize energy
simulation = Simulation(pdb.topology, system, integrator)
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('stochastic_LE.dcd', 10))
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy(tolerance=0.001)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('minimized.cif', 'w')) # save minimized file

# 4. Run md simulation
simulation.context.setVelocitiesToTemperature(310, 0)
simulation.step(10000)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.cif', 'w')) # save minimized file
PDBFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.pdb', 'w')) # save minimized file
df = DCDFile(open("after_sim.dcd", "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,159.43203735351562,755.1473015267402,240.02706116252395
20,138.3152618408203,756.6316171213984,249.13354844566172
30,157.8941650390625,756.0391945242882,241.0060681852224
40,169.38449096679688,744.7406327724457,231.8239134447639
50,159.6554718017578,766.5091916844249,244.51499516491504
60,161.51193237304688,783.3031955622137,250.5336668969935
70,176.68765258789062,805.5879553854465,253.39805864169364
80,203.76632690429688,837.7395208999515,255.4420404549963
90,206.0120849609375,863.2368396613747,264.8106165500821
100,178.2198486328125,860.7066193073988,274.98925023295607
110,180.57846069335938,861.5901501914486,274.3949068930013
120,181.4359130859375,863.5769528634846,274.8499474885744
130,182.4939727783203,885.2192020565271,283.1437798098142
140,180.57894897460938,899.019144166261,289.47569257305094
150,208.79693603515625,899.0253336876631,278.10852563830554
160,221.42343139648438,941.6336359120905,290.

In [45]:
original_min = 604886
original_max = 983769
new_min = 1
new_max = 200


df_scaled = middle_points.map(lambda x: scale_value(x, original_min, original_max, new_min, new_max))

In [48]:
df_scaled


In [49]:
df_scaled

,middle1,middle2
0,1,178
1,92,117
2,118,140
3,120,134
4,137,158
5,137,177
6,140,146
7,139,152
8,139,185
9,140,199


Modeling

In [3]:
traj = md.load("..\\after_sim.cif")

positions = traj.xyz

mesh = pv.PolyData(positions[0])

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions[0])

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:60519/index.html?ui=P_0x2264ed5ab80_0&reconnect=auto" class="pyvis…